source: https://towardsdatascience.com/deploy-your-machine-learning-model-as-a-rest-api-4fe96bf8ddcc

## Part 1 - Train ML model and save it

In [3]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

In [4]:
def train_model():
    iris_df = datasets.load_iris()

    x = iris_df.data
    y = iris_df.target

    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
    dt = DecisionTreeClassifier().fit(X_train, y_train)
    preds = dt.predict(X_test)

    accuracy = accuracy_score(y_test, preds)
    joblib.dump(dt, 'iris-model.model')
    print('Model Training Finished.\n\tAccuracy obtained: {}'.format(accuracy))

## Part 2 - Create a app.py script and execute it using terminal / command prompt

In [5]:
import os
from flask import Flask, jsonify, request
from flask_restful import Api, Resource

In [6]:
app = Flask(__name__)
api = Api(app)

In [7]:
if not os.path.isfile('iris-model.model'):
    train_model()

model = joblib.load('iris-model.model')

Model Training Finished.
	Accuracy obtained: 0.8157894736842105


In [8]:
class MakePrediction(Resource):
    @staticmethod
    def post():
        posted_data = request.get_json()
        sepal_length = posted_data['sepal_length']
        sepal_width = posted_data['sepal_width']
        petal_length = posted_data['petal_length']
        petal_width = posted_data['petal_width']

        prediction = model.predict([[sepal_length, sepal_width, petal_length, petal_width]])[0]
        if prediction == 0:
            predicted_class = 'Iris-setosa'
        elif prediction == 1:
            predicted_class = 'Iris-versicolor'
        else:
            predicted_class = 'Iris-virginica'

        return jsonify({
            'Prediction': predicted_class
        })

In [9]:
api.add_resource(MakePrediction, '/predict')

In [10]:
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

D:\Users\PShirbhate\AppData\Local\Continuum\anaconda3\envs\flask_env\lib\site-packages\IPython\core\interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Part 3 - Test using Postman - localhost:5000/predict

Now open Postman and do the following:
- Change the method to POST
- Enter localhost:5000/predict as the URL
- Inside the Body tab choose JSON
- Enter some JSON for prediction

{
    "sepal_length":2,
    "sepal_width":4.2,
    "petal_length":1.5,
    "petal_width":2.3
}